In [69]:
import tempfile
import tvm
from tvm.script import ir as I
from tvm.script import relax as R
from tvm.script import tir as T
from tvm.target import Target

from tvm import relax
from tvm import dlight as dl

import numpy as np
import torch

# Dimensions
batch = 1
in_channels = 128
out_channels = 128
input_width = 220500
kernel_size = 7
stride = 1
dilation = 1
padding = 3
output_width = (input_width + 2 * padding - dilation * (kernel_size - 1) - 1) // stride + 1

@I.ir_module
class Module:
    @T.prim_func
    def conv1d_im2col(A: T.Buffer((1, 128, 220500), "float32"),
                    W: T.Buffer((128, 128, 7), "float32"),
                    B: T.Buffer((1, 128, 220500), "float32")):

        col_buffer = T.alloc_buffer((batch, in_channels, kernel_size, output_width), dtype="float32", scope="global")

        # im2col
        for b, c, i, k in T.grid(batch, in_channels, output_width, kernel_size):
            with T.block("im2col"):
                v_b, v_c, v_i, v_k = T.axis.remap("SSSS", [b, c, i, k])
                padded_i = v_i + v_k * dilation - padding
                if (padded_i >= 0) and (padded_i < input_width):
                    col_buffer[v_b, v_c, v_k, v_i] = A[v_b, v_c, padded_i]
                else:
                    col_buffer[v_b, v_c, v_k, v_i] = 0.0

        # gemm
        for b, oc, i, c, k in T.grid(batch, out_channels, output_width, in_channels, kernel_size):
            with T.block("conv1d"):
                v_b, v_oc, v_i, v_c, v_k = T.axis.remap("SSSRR", [b, oc, i, c, k])
                with T.init():
                    B[v_b, v_oc, v_i] = 0.0
                B[v_b, v_oc, v_i] += col_buffer[v_b, v_c, v_k, v_i] * W[v_oc, v_c, v_k]

    # Relax function that invokes TIR
    @R.function
    def main(A: R.Tensor((1, 128, 220500), dtype="float32"), 
                W: R.Tensor((128, 128, 7), dtype="float32")) -> R.Tensor((1, 128, 220500), dtype="float32"):
        cls = Module
        with R.dataflow():
            # Call the TIR function using Relax call_tir
            lv = R.call_tir(cls.conv1d_im2col, (A, W), out_sinfo=R.Tensor((1, 128, 220500), dtype="float32"))
            gv: R.Tensor((1, 128, 220500), dtype="float32") = lv
            R.output(gv)
        return gv

target = Target.from_device("metal")
mod = Module

trials = 2000
with target, tempfile.TemporaryDirectory() as tmp_dir:
    seq = tvm.transform.Sequential(
        [
            relax.get_pipeline("zero"),
            relax.transform.MetaScheduleTuneTIR(work_dir=tmp_dir, max_trials_global=trials),
            relax.transform.MetaScheduleApplyDatabase(work_dir=tmp_dir),
        ]
    )

    mod = seq(mod)

mod.show()

device = tvm.metal()
ex = relax.build(mod, target)
vm = relax.VirtualMachine(ex, device=device)

A = torch.rand((1, 128, 220500))
W = torch.rand((128, 128, 7))

res_np = torch.conv1d(A, W, padding=3).numpy()
print(res_np)

A_tvm = tvm.nd.array(A.numpy(), device=device)
W_tvm = tvm.nd.array(W.numpy(), device=device)

out = vm["main"](A_tvm, W_tvm)
print(out.numpy())

2024-10-26 16:48:46 [INFO] Logging directory: /var/folders/50/mzqbqxqj5fddcby2mg3h334c0000gp/T/tmp6a4df409/logs
2024-10-26 16:48:46 [INFO] LocalBuilder: max_workers = 24


[16:48:46] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[16:48:46] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[16:48:46] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`


2024-10-26 16:48:47 [INFO] LocalRunner: max_workers = 1
2024-10-26 16:48:48 [INFO] [task_scheduler.cc:159] Initializing Task #0: "main"
2024-10-26 16:48:48 [INFO] [task_scheduler.cc:180] TaskScheduler picks Task #0: "main"
2024-10-26 16:50:03 [INFO] [task_scheduler.cc:193] Sending 64 sample(s) to builder


[16:50:04] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[16:50:04] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[16:50:04] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[16:50:04] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[16:50:04] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=

2024-10-26 16:50:09 [INFO] [task_scheduler.cc:195] Sending 64 sample(s) to runner
2024-10-26 16:51:51 [DEBUG] XGB iter   0: tr-p-rmse: 0.536934	tr-a-peak@32: 1.000000	tr-rmse: 0.615243	tr-rmse: 0.615243
2024-10-26 16:51:51 [DEBUG] XGB iter  25: tr-p-rmse: 0.134948	tr-a-peak@32: 1.000000	tr-rmse: 0.591185	tr-rmse: 0.591185
2024-10-26 16:51:51 [DEBUG] XGB iter  50: tr-p-rmse: 0.134948	tr-a-peak@32: 1.000000	tr-rmse: 0.591185	tr-rmse: 0.591185
2024-10-26 16:51:51 [DEBUG] XGB stopped. Best iteration: [11] tr-p-rmse:0.13495	tr-a-peak@32:1.00000	tr-rmse:0.59119	tr-rmse:0.59119 
2024-10-26 16:51:51 [INFO] [task_scheduler.cc:237] [Updated] Task #0: "main"


...

2024-10-26 18:00:22 [DEBUG] [task_scheduler.cc:318] 
 ID | Name |        FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
----------------------------------------------------------------------------------------------------------
  0 | main | 50972544000 |      1 |      3951.0633 |   12900.9688 |            12900.9688 |   2000 |      
----------------------------------------------------------------------------------------------------------
Total trials: 2000
Total latency (us): 12901


Total trials: 2000
Total latency (us): 12901

2024-10-26 18:00:22 [INFO] [task_scheduler.cc:260] Task #0 has finished. Remaining task(s): 0


,Name,FLOP,Weight,Speed (GFLOPS),Latency (us),Weighted Latency (us),Trials,Done
0,main,50972544000,1,3951.0633,12900.9688,12900.9688,2000,Y


2024-10-26 18:00:22 [DEBUG] [task_scheduler.cc:318] 
 ID | Name |        FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
----------------------------------------------------------------------------------------------------------
  0 | main | 50972544000 |      1 |      3951.0633 |   12900.9688 |            12900.9688 |   2000 |    Y 
----------------------------------------------------------------------------------------------------------
Total trials: 2000
Total latency (us): 12901


Total trials: 2000
Total latency (us): 12901



[18:00:23] /Users/cfruan/Documents/tvm-unity/src/relax/transform/meta_schedule.cc:119: Warning: Creating JSONDatabase. Workload at: /var/folders/50/mzqbqxqj5fddcby2mg3h334c0000gp/T/tmp6a4df409/database_workload.json, Tuning records at: /var/folders/50/mzqbqxqj5fddcby2mg3h334c0000gp/T/tmp6a4df409/database_tuning_record.json


[[[125.62249  155.16917  187.27216  ... 200.26817  155.66434  128.04039 ]
  [123.18549  151.71077  180.83754  ... 192.05782  157.52486  126.9617  ]
  [121.7238   154.5874   185.50322  ... 196.46666  156.68903  119.301285]
  ...
  [124.33756  153.0014   183.11923  ... 189.70364  158.41801  128.54935 ]
  [117.28619  150.46982  183.40135  ... 188.33183  153.7275   122.42524 ]
  [122.118935 153.12526  189.416    ... 188.88391  161.4157   129.76212 ]]]
[[[125.62248  155.16916  187.27205  ... 200.26819  155.66438  128.04039 ]
  [123.18547  151.71082  180.83755  ... 192.05785  157.52489  126.9617  ]
  [121.7238   154.58737  185.50327  ... 196.4667   156.68909  119.301254]
  ...
  [124.33756  153.0014   183.11934  ... 189.70366  158.41808  128.54932 ]
  [117.28616  150.46985  183.40135  ... 188.3318   153.7275   122.42522 ]
  [122.11893  153.12523  189.41599  ... 188.88397  161.41566  129.76215 ]]]


In [67]:
from tvm.relax.frontend import nn

conv1d = nn.Conv1D(128, 128, 7, dilation=1, padding=3)
mod, params = conv1d.export_tvm(
    {"forward": {"x": nn.spec.Tensor((1, 128, 220500), "float32")}}
)

with target:
    mod = relax.get_pipeline("zero")(mod)
mod.show()

In [64]:
import tempfile
from typing import Optional, Tuple, Union
from tvm import tir
from tvm.relax.frontend.nn import Tensor, op
import torch


def get_conv1d(
    x: Tensor,
    weight: Tensor,
    bias: Optional[Tensor] = None,
    stride: Optional[int] = 1,
    padding: Optional[int] = 0,
    dilation: Optional[int] = 1,
):
    batch, in_channels, input_width = x.shape
    out_channels, _, kernel_size = weight.shape
    output_width = (
        input_width + 2 * padding - dilation * (kernel_size - 1) - 1
    ) // stride + 1

    @T.prim_func
    def conv1d_im2col(
        A: T.Buffer((1, 128, 220500), "float32"),
        W: T.Buffer((128, 128, 7), "float32"),
        B: T.Buffer((1, 128, 220500), "float32"),
    ):
        # A is the input tensor of shape [batch, in_channels, width]
        # W is the weight tensor of shape [out_channels, in_channels, kernel_size]
        # B is the output tensor of shape [batch, out_channels, output_width]

        # Temporary buffer for im2col result
        col_buffer = T.alloc_buffer(
            (batch, in_channels, kernel_size, output_width),
            dtype="float32",
            scope="global",
        )

        # im2col: Extract patches from input tensor A
        for b, c, i, k in T.grid(batch, in_channels, output_width, kernel_size):
            with T.block("im2col"):
                v_b, v_c, v_i, v_k = T.axis.remap("SSSS", [b, c, i, k])
                padded_i = v_i * stride + v_k * dilation - padding
                if (padded_i >= 0) and (padded_i < input_width):
                    col_buffer[v_b, v_c, v_k, v_i] = A[v_b, v_c, padded_i]
                else:
                    col_buffer[v_b, v_c, v_k, v_i] = 0.0

        # Perform matrix multiplication between col_buffer and the weight matrix W
        for b, oc, i in T.grid(batch, out_channels, output_width):
            with T.block("conv1d"):
                v_b, v_oc, v_i = T.axis.remap("SSS", [b, oc, i])
                # Initialize B for reduction
                with T.init():
                    B[v_b, v_oc, v_i] = 0.0
                for c, k in T.grid(in_channels, kernel_size):
                    B[v_b, v_oc, v_i] += col_buffer[v_b, c, k, v_i] * W[v_oc, c, k]

    return conv1d_im2col


mod = tvm.IRModule()
mod["conv1d_im2col"] = get_conv1d(
    Tensor.placeholder((1, 128, 220500), "float32"),
    Tensor.placeholder((128, 128, 7), "float32"),
    stride=1,
    padding=3,
    dilation=1,
)
target = Target.from_device("metal")

trials = 2000
with target, tempfile.TemporaryDirectory() as tmp_dir:
    seq = tvm.transform.Sequential(
        [
            # tvm.relax.transform.LegalizeOps(),
            # tvm.relax.transform.AnnotateTIROpPattern(),
            # tvm.relax.transform.FoldConstant(),
            # tvm.relax.transform.FuseOps(),
            # tvm.relax.transform.FuseTIR(),
            # dl.ApplyDefaultSchedule(
            #     dl.gpu.Matmul(),
            #     dl.gpu.GEMV(),
            #     dl.gpu.Reduction(),
            #     dl.gpu.GeneralReduction(),
            #     dl.gpu.Fallback(),
            # ),

            relax.get_pipeline("zero"),
            relax.transform.MetaScheduleTuneTIR(work_dir=tmp_dir, max_trials_global=trials),
            relax.transform.MetaScheduleApplyDatabase(work_dir=tmp_dir),
        ]
    )

    mod = seq(mod)

mod.show()

tvm_mod = tvm.build(mod, target="metal")

A = torch.rand((1, 128, 220500))
W = torch.rand((128, 128, 7))

res_np = torch.conv1d(A, W, padding=3).numpy()
print(res_np)

A_tvm = tvm.nd.array(A.numpy(), device=device)
W_tvm = tvm.nd.array(W.numpy(), device=device)
B_tvm = tvm.nd.array(np.zeros((1, 128, 220500), dtype=np.float32), device=device)

tvm_mod["conv1d_im2col"](A_tvm, W_tvm, B_tvm)
print(B_tvm.numpy())

res = tvm_mod.time_evaluator("conv1d_im2col", device, number=3, repeat=10, min_repeat_ms=100)(A_tvm, W_tvm, B_tvm)
print(res)

# # GPU Schedule
# sch = tir.Schedule(conv1d_im2col)

# # Get loops to bind to threads and blocks
# print(sch.get_loops(sch.get_block("conv1d")))
# block_b, block_oc, block_i = sch.get_loops(sch.get_block("conv1d"))
# sch.bind(block_b, "blockIdx.x")
# sch.bind(block_oc, "threadIdx.x")

# block_b, block_c, block_i, block_k = sch.get_loops(sch.get_block("im2col"))
# sch.bind(block_b, "blockIdx.x")
# sch.bind(block_c, "threadIdx.x")

# sch.show()

# mod = tvm.build(sch.mod, target="metal")

# A = torch.rand((1, 128, 220500))
# W = torch.rand((128, 128, 7))

# res_np = torch.conv1d(A, W, padding=3).numpy()
# print(res_np)

# device = tvm.metal()
# A_tvm = tvm.nd.array(A.numpy(), device=device)
# W_tvm = tvm.nd.array(W.numpy(), device=device)
# B_tvm = tvm.nd.array(np.zeros((1, 128, 220500), dtype=np.float32), device=device)

# mod(A_tvm, W_tvm, B_tvm)
# print(B_tvm.numpy())

2024-10-26 13:02:48 [INFO] Logging directory: /var/folders/50/mzqbqxqj5fddcby2mg3h334c0000gp/T/tmpl9gamhz4/logs
2024-10-26 13:02:54 [INFO] LocalBuilder: max_workers = 24


[13:02:54] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[13:02:54] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[13:02:54] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`


2024-10-26 13:02:55 [INFO] LocalRunner: max_workers = 1
2024-10-26 13:02:56 [INFO] [task_scheduler.cc:159] Initializing Task #0: "main"
2024-10-26 13:02:56 [INFO] [task_scheduler.cc:180] TaskScheduler picks Task #0: "main"
2024-10-26 13:03:00 [INFO] [task_scheduler.cc:193] Sending 1 sample(s) to builder


[13:03:00] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[13:03:00] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[13:03:00] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`


2024-10-26 13:03:00 [INFO] [task_scheduler.cc:195] Sending 1 sample(s) to runner
2024-10-26 13:03:03 [DEBUG] XGB iter   0: tr-p-rmse: 0.000000	tr-a-peak@32: 1.000000	tr-rmse: 0.500000	tr-rmse: 0.500000
2024-10-26 13:03:03 [DEBUG] XGB iter  25: tr-p-rmse: 0.000000	tr-a-peak@32: 1.000000	tr-rmse: 0.500000	tr-rmse: 0.500000
2024-10-26 13:03:03 [DEBUG] XGB iter  50: tr-p-rmse: 0.000000	tr-a-peak@32: 1.000000	tr-rmse: 0.500000	tr-rmse: 0.500000
2024-10-26 13:03:03 [DEBUG] XGB stopped. Best iteration: [0] tr-p-rmse:0.00000	tr-a-peak@32:1.00000	tr-rmse:0.50000	tr-rmse:0.50000 
2024-10-26 13:03:03 [INFO] [task_scheduler.cc:237] [Updated] Task #0: "main"


...


Total trials: 1
Total latency (us): 340030

2024-10-26 13:03:18 [DEBUG] [task_scheduler.cc:318] 
 ID | Name |        FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
----------------------------------------------------------------------------------------------------------
  0 | main | 50972544000 |      1 |       149.9062 |  340029.5557 |           340029.5557 |      1 |      
----------------------------------------------------------------------------------------------------------
Total trials: 1
Total latency (us): 340030

2024-10-26 13:03:18 [INFO] [task_scheduler.cc:180] TaskScheduler picks Task #0: "main"
2024-10-26 13:03:22 [INFO] [task_scheduler.cc:260] Task #0 has finished. Remaining task(s): 0


,Name,FLOP,Weight,Speed (GFLOPS),Latency (us),Weighted Latency (us),Trials,Done
0,main,50972544000,1,149.9062,340029.5557,340029.5557,1,Y


2024-10-26 13:03:22 [DEBUG] [task_scheduler.cc:318] 
 ID | Name |        FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
----------------------------------------------------------------------------------------------------------
  0 | main | 50972544000 |      1 |       149.9062 |  340029.5557 |           340029.5557 |      1 |    Y 
----------------------------------------------------------------------------------------------------------
Total trials: 1
Total latency (us): 340030


Total trials: 1
Total latency (us): 340030



[13:03:22] /Users/cfruan/Documents/tvm-unity/src/relax/transform/meta_schedule.cc:119: Warning: Creating JSONDatabase. Workload at: /var/folders/50/mzqbqxqj5fddcby2mg3h334c0000gp/T/tmpl9gamhz4/database_workload.json, Tuning records at: /var/folders/50/mzqbqxqj5fddcby2mg3h334c0000gp/T/tmpl9gamhz4/database_tuning_record.json


[[[123.38553  156.4974   182.82404  ... 178.33762  152.15555  124.374954]
  [126.65324  162.63055  195.49226  ... 190.29411  156.49454  132.81216 ]
  [128.82669  165.09639  199.52286  ... 191.69012  160.10316  131.97855 ]
  ...
  [130.43837  162.78171  194.11246  ... 188.41476  155.70473  123.31559 ]
  [124.17619  165.63484  192.64189  ... 190.44757  155.91843  125.4309  ]
  [137.23804  167.51942  196.60655  ... 189.99191  156.77925  125.08389 ]]]
[[[123.38553  156.4974   182.82404  ... 178.33762  152.15555  124.374954]
  [126.65324  162.63055  195.49226  ... 190.29411  156.49454  132.81216 ]
  [128.82669  165.09639  199.52286  ... 191.69012  160.10316  131.97855 ]
  ...
  [130.43837  162.78171  194.11246  ... 188.41476  155.70473  123.31559 ]
  [124.17619  165.63484  192.64189  ... 190.44757  155.91843  125.4309  ]
  [137.23804  167.51942  196.60655  ... 189.99191  156.77925  125.08389 ]]]
Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  340.8

In [1]:
import tvm

from tvm import relax
from tvm.relax.frontend import nn

from typing import Optional
from tvm import te
from tvm import dlight as dl
from tvm.target import Target
import numpy as np
import tempfile

from mlc_dac.layers import WNConv1d

# conv1d = WNConv1d(128, 128, 7, dilation=1, padding=3)

conv1d = nn.Conv1D(128, 128, 7, dilation=1, padding=3)
mod, params = conv1d.export_tvm(
    {"forward": {"x": nn.spec.Tensor((1, 128, 220500), "float32")}}
)

trials = 2000
target = Target.from_device("metal")

with target, tempfile.TemporaryDirectory() as tmp_dir:
    seq = tvm.transform.Sequential(
        [
            # tvm.relax.transform.LegalizeOps(),
            # tvm.relax.transform.AnnotateTIROpPattern(),
            # tvm.relax.transform.FoldConstant(),
            # tvm.relax.transform.FuseOps(),
            # tvm.relax.transform.FuseTIR(),
            # dl.ApplyDefaultSchedule(
            #     # dl.gpu.Matmul(),
            #     dl.gpu.GEMV(),
            #     dl.gpu.Reduction(),
            #     dl.gpu.GeneralReduction(),
            #     dl.gpu.Fallback(),
            # ),

            relax.get_pipeline("zero"),
            # relax.transform.MetaScheduleTuneTIR(work_dir=tmp_dir, max_trials_global=trials),
            # relax.transform.MetaScheduleApplyDatabase(work_dir=tmp_dir),
        ]
    )

    mod = seq(mod)

mod.show()

[23:24:34] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[23:24:34] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[23:24:34] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
